In [ ]:
!pip install geopandas mapclassify  # mapclassify для визуализации
!pip install osmnx  # устанавливаем модуль osmnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 3.6 MB/s eta 0:00:00


In [ ]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import os

Добавляем переменную data_path, в которую будем складывать результаты работы в блокноте. До этого необходимо создать папку data в директории.

In [ ]:
data_path = '/content/data' # из этой папки достаём geojson файлы

Загружаем границы территории

In [ ]:
boarders = gpd.read_file(os.path.join(data_path, 'boarders.geojson')) # считываем в переменную границы территории исследования

In [ ]:
boarders.explore(tiles="CartoDB positron")

Выгрузка слоёв из OSM, их фильтрация и экспорт в .geojson

In [ ]:
tags = {'building': True}  # получить все здания, аналогично building=*
buildings = ox.features.features_from_polygon(boarders.unary_union, tags)
buildings.head()

<ipython-input-6-38926a48520c>:2: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  buildings = ox.features.features_from_polygon(boarders.unary_union, tags)


geometry  \
element  id                                                            
relation 10354551  MULTIPOLYGON (((30.26828 59.96254, 30.26829 59...   
         14893079  POLYGON ((30.26943 59.96383, 30.26937 59.96386...   
         14893100  POLYGON ((30.27021 59.9627, 30.26996 59.96257,...   
way      39065438  POLYGON ((30.27395 59.96303, 30.27418 59.96286...   
         61038286  POLYGON ((30.27221 59.96154, 30.27203 59.96143...   

                         addr:city addr:housenumber addr:postcode  \
element  id                                                         
relation 10354551              NaN               45           NaN   
         14893079  Санкт-Петербург               50           NaN   
         14893100              NaN          50 литГ           NaN   
way      39065438  Санкт-Петербург               51        197110   
         61038286  Санкт-Петербург               43           NaN   

                               addr:street    building building:levels  \
element  id                                                              
relation 10354551         Ждановская улица  apartments               1   
         14893079         Пионерская улица  apartments              16   
         14893100         Пионерская улица         yes             NaN   
way      39065438  улица Красного Курсанта  apartments               6   
         61038286         Ждановская улица         yes               2   

                  addr:country        old_name roof:shape  ... substation  \
element  id                                                ...              
relation 10354551          NaN             NaN        NaN  ...        NaN   
         14893079          NaN             NaN        NaN  ...        NaN   
         14893100          NaN             NaN        NaN  ...        NaN   
way      39065438          NaN             NaN        NaN  ...        NaN   
         61038286           RU  Дом Леонтьевых        NaN  ...        NaN   

                  abandoned abandoned:amenity opening_hours  \
element  id                                                   
relation 10354551       NaN               NaN           NaN   
         14893079       NaN               NaN           NaN   
         14893100       yes               NaN           NaN   
way      39065438       NaN               NaN           NaN   
         61038286       NaN               NaN           NaN   

                  service:vehicle:oil_change website abandoned:shop  \
element  id                                                           
relation 10354551                        NaN     NaN            NaN   
         14893079                        NaN     NaN            NaN   
         14893100                        NaN     NaN            NaN   
way      39065438                        NaN     NaN            NaN   
         61038286                        NaN     NaN            NaN   

                           type building:min_level building:year_built  
element  id                                                             
relation 10354551  multipolygon                  0                2017  
         14893079  multipolygon                NaN                 NaN  
         14893100  multipolygon                NaN                 NaN  
way      39065438           NaN                NaN                 NaN  
         61038286           NaN                NaN                 NaN  

[5 rows x 38 columns]

In [ ]:
buildings.explore(tiles="CartoDB positron")

In [ ]:
# Выбор нужных столбцов (включая geometry)
columns_to_keep_buildings = ['building', 'building:levels', 'geometry']

# Создаем новый датафрейм с выбранными столбцами
buildings_f = buildings[columns_to_keep_buildings]

# и выводим его посмотреть
buildings_f.head()

building building:levels  \
element  id                                     
relation 10354551  apartments               1   
         14893079  apartments              16   
         14893100         yes             NaN   
way      39065438  apartments               6   
         61038286         yes               2   

                                                            geometry  
element  id                                                           
relation 10354551  MULTIPOLYGON (((30.26828 59.96254, 30.26829 59...  
         14893079  POLYGON ((30.26943 59.96383, 30.26937 59.96386...  
         14893100  POLYGON ((30.27021 59.9627, 30.26996 59.96257,...  
way      39065438  POLYGON ((30.27395 59.96303, 30.27418 59.96286...  
         61038286  POLYGON ((30.27221 59.96154, 30.27203 59.96143...

In [ ]:
# Сохраняем отфильтрованный датафрейм обратно в GeoJSON
buildings_f.to_file('buildings.geojson', driver='GeoJSON')

In [ ]:
tags = {"highway": ["primary", "secondary", "tertiary", "unclassified", "living_street", "residential", "service", "motorway", "trunk"]}  # получить улицы и проезды
highway = ox.features.features_from_polygon(boarders.unary_union, tags)
highway.head()

<ipython-input-15-bd90fa0ab734>:2: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  highway = ox.features.features_from_polygon(boarders.unary_union, tags)


geometry  \
element id                                                             
way     8145675    LINESTRING (30.27425 59.96228, 30.27444 59.962...   
        172613529  LINESTRING (30.27637 59.9614, 30.27592 59.9616...   
        173093084  LINESTRING (30.27452 59.96247, 30.27444 59.962...   
        182180782  LINESTRING (30.26561 59.96389, 30.26858 59.964...   
        182180785  LINESTRING (30.27261 59.96221, 30.2728 59.9621...   

                       highway foot                  int_name lanes  lit  \
element id                                                                 
way     8145675    residential   no  Ulitsa Krasnogo Kursanta     1  yes   
        172613529  residential   no                       NaN     1  yes   
        173093084  residential   no  Ulitsa Krasnogo Kursanta     2  yes   
        182180782      service  NaN                       NaN   NaN  NaN   
        182180785      service  NaN                       NaN   NaN  NaN   

                   maxspeed                     name                  name:ru  \
element id                                                                      
way     8145675    RU:urban  улица Красного Курсанта  улица Красного Курсанта   
        172613529  RU:urban  улица Красного Курсанта  улица Красного Курсанта   
        173093084  RU:urban  улица Красного Курсанта  улица Красного Курсанта   
        182180782       NaN                      NaN                      NaN   
        182180785       NaN                      NaN                      NaN   

                  oneway  ...  psv psv:lanes bicycle cycleway:right  \
element id                ...                                         
way     8145675      yes  ...  NaN       NaN     NaN            NaN   
        172613529    yes  ...  NaN       NaN     NaN            NaN   
        173093084    NaN  ...  NaN       NaN     NaN            NaN   
        182180782    NaN  ...  NaN       NaN     NaN            NaN   
        182180785    NaN  ...  NaN       NaN     NaN            NaN   

                  source:maxspeed parking:lane:right sidewalk:left  \
element id                                                           
way     8145675               NaN                NaN           NaN   
        172613529             NaN                NaN           NaN   
        173093084             NaN                NaN           NaN   
        182180782             NaN                NaN           NaN   
        182180785             NaN                NaN           NaN   

                  parking:lane:both fixme covered  
element id                                         
way     8145675                 NaN   NaN     NaN  
        172613529               NaN   NaN     NaN  
        173093084               NaN   NaN     NaN  
        182180782               NaN   NaN     NaN  
        182180785               NaN   NaN     NaN  

[5 rows x 42 columns]

In [ ]:
highway.explore(tiles="CartoDB positron")

In [ ]:
# Сброс индексов в геодатафрейме с удалением старых индексов и сохранением изменений в исходном геодатафрейме
highway.reset_index(drop=True, inplace=True)

# Выбор нужных столбцов (включая geometry)
columns_to_keep_highway = [ 'highway', 'name', 'living_street', 'geometry']

# Создаем новый датафрейм с выбранными столбцами
highway_f = highway[columns_to_keep_highway]

# и выводим его посмотреть
highway_f

,highway,name,living_street,geometry
0,residential,улица Красного Курсанта,NaN,"LINESTRING (30.27425 59.96228, 30.27444 59.962..."
1,residential,улица Красного Курсанта,NaN,"LINESTRING (30.27637 59.9614, 30.27592 59.9616..."
2,residential,улица Красного Курсанта,NaN,"LINESTRING (30.27452 59.96247, 30.27444 59.962..."
3,service,NaN,NaN,"LINESTRING (30.26561 59.96389, 30.26858 59.964..."
4,service,NaN,yes,"LINESTRING (30.27261 59.96221, 30.2728 59.9621..."
...,...,...,...,...
62,service,NaN,NaN,"LINESTRING (30.27618 59.96309, 30.27613 59.963..."
63,service,NaN,NaN,"LINESTRING (30.27478 59.96306, 30.27474 59.963..."
64,service,NaN,NaN,"LINESTRING (30.27655 59.96314, 30.27659 59.96322)"
65,residential,Гдовская улица,NaN,"LINESTRING (30.27414 59.96277, 30.27424 59.9628)"


In [ ]:
# Сохраняем отфильтрованный датафрейм обратно в GeoJSON
highway_f.to_file('highway.geojson', driver='GeoJSON')

In [ ]:
tags = {"amenity": ["parking", "parking_space"]}  # получить парковки
parking = ox.features.features_from_polygon(boarders.unary_union, tags)  # district.unary_union - возвращает объединённую геометрию
parking.head()

<ipython-input-19-9ad61bc8e467>:2: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  parking = ox.features.features_from_polygon(boarders.unary_union, tags)  # district.unary_union - возвращает объединённую геометрию


,,geometry,amenity,fee,layer,parking,parking:orientation
element,id,,,,,,
node,11039783520,POINT (30.26875 59.96288),parking,yes,-1,underground,NaN
way,978740323,"POLYGON ((30.27373 59.96297, 30.2738 59.963, 3...",parking,NaN,NaN,street_side,perpendicular


In [ ]:
parking.explore(tiles="CartoDB positron")

In [ ]:
# Сброс индексов в геодатафрейме с удалением старых индексов и сохранением изменений в исходном геодатафрейме
parking.reset_index(drop=True, inplace=True)

# Выбор нужных столбцов (включая geometry)
columns_to_keep_parking = [ 'amenity', 'geometry']

# Создаем новый датафрейм с выбранными столбцами
parking_f = parking[columns_to_keep_parking]

# и выводим его посмотреть
parking_f

,amenity,geometry
0,parking,POINT (30.26875 59.96288)
1,parking,"POLYGON ((30.27373 59.96297, 30.2738 59.963, 3..."


In [ ]:
# Сохраняем отфильтрованный датафрейм обратно в GeoJSON
parking_f.to_file('parking.geojson', driver='GeoJSON')

In [ ]:
tags = {"amenity": ["recycling", "waste_disposal"], "landuse": ["landfill"]}  # получить геометрию
recycling = ox.features.features_from_polygon(boarders.unary_union, tags)  # district.unary_union - возвращает объединённую геометрию
recycling.head()

<ipython-input-23-20508085ccec>:2: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  recycling = ox.features.features_from_polygon(boarders.unary_union, tags)  # district.unary_union - возвращает объединённую геометрию


,,geometry,amenity
element,id,,
node,8904836385,POINT (30.27506 59.96312),waste_disposal


In [ ]:
recycling.explore(tiles="CartoDB positron")

In [ ]:
# Сброс индексов в геодатафрейме с удалением старых индексов и сохранением изменений в исходном геодатафрейме
recycling.reset_index(drop=True, inplace=True)

# Выбор нужных столбцов (включая geometry)
columns_to_keep_recycling = [ 'amenity', 'geometry']

# Создаем новый датафрейм с выбранными столбцами
recycling_f = recycling[columns_to_keep_recycling]

# и выводим его посмотреть
recycling_f

,amenity,geometry
0,waste_disposal,POINT (30.27506 59.96312)


In [ ]:
# Сохраняем отфильтрованный датафрейм обратно в GeoJSON
recycling_f.to_file('recycling.geojson', driver='GeoJSON')

In [ ]:
# для позиций, которые есть не во всех случаях
tags = {"man_made": ["works"], "landuse": ["industrial"]} # получить геометрию
try:
    industrial = ox.features.features_from_polygon(boarders.unary_union, tags)  # district.unary_union - возвращает объединённую геометрию
    industrial.head()
    industrial.reset_index(drop=True, inplace=True) # Сброс индексов в геодатафрейме с удалением старых индексов и сохранением изменений в исходном геодатафрейме
    columns_to_keep_industrial = [ 'amenity', 'geometry'] # Выбор нужных столбцов (включая geometry)
    industrial_f = industrial[columns_to_keep_industrial] # Создаем новый датафрейм с выбранными столбцами
    industrial_f # и выводим его посмотреть
    industrial_f.to_file('industrial.geojson', driver='GeoJSON') # Сохраняем отфильтрованный датафрейм обратно в GeoJSON

except ox._errors.InsufficientResponseError:
    print("Нет данных по указанному запросу")

<ipython-input-26-8f28b1fd6596>:4: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  industrial = ox.features.features_from_polygon(boarders.unary_union, tags)  # district.unary_union - возвращает объединённую геометрию


Нет данных по указанному запросу


In [ ]:
tags = {"building": ["service", "transformer_tower"]}  # получить трансформаторные подстанции и тп
electric = ox.features.features_from_polygon(boarders.unary_union, tags)  # district.unary_union - возвращает объединённую геометрию
electric.head()

<ipython-input-27-b22151bd621d>:2: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  electric = ox.features.features_from_polygon(boarders.unary_union, tags)  # district.unary_union - возвращает объединённую геометрию


geometry  \
element id                                                             
way     182180784  POLYGON ((30.27324 59.96316, 30.27339 59.96322...   
        256927648  POLYGON ((30.27249 59.96171, 30.27242 59.96175...   
        337390187  POLYGON ((30.27517 59.96315, 30.27525 59.96319...   
        552294493  POLYGON ((30.27191 59.96268, 30.27206 59.96274...   

                  addr:housenumber              addr:street building  \
element id                                                             
way     182180784            40 к2  улица Красного Курсанта  service   
        256927648              NaN                      NaN  service   
        337390187           5 литТ           Гдовская улица  service   
        552294493             6 к2      Новоладожская улица  service   

                  building:levels       power roof:shape          substation  
element id                                                                    
way     182180784             NaN         NaN        NaN                 NaN  
        256927648               1  substation       flat  minor_distribution  
        337390187               2  substation       flat                 NaN  
        552294493             NaN         NaN        NaN                 NaN

In [ ]:
electric.explore(tiles="CartoDB positron")

In [ ]:
# Сброс индексов в геодатафрейме с удалением старых индексов и сохранением изменений в исходном геодатафрейме
electric.reset_index(drop=True, inplace=True)

# Выбор нужных столбцов (включая geometry)
columns_to_keep_electric = [ 'building', 'geometry']

# Создаем новый датафрейм с выбранными столбцами
electric_f = electric[columns_to_keep_electric]

# и выводим его посмотреть
electric_f

,building,geometry
0,service,"POLYGON ((30.27324 59.96316, 30.27339 59.96322..."
1,service,"POLYGON ((30.27249 59.96171, 30.27242 59.96175..."
2,service,"POLYGON ((30.27517 59.96315, 30.27525 59.96319..."
3,service,"POLYGON ((30.27191 59.96268, 30.27206 59.96274..."


In [ ]:
# Сохраняем отфильтрованный датафрейм обратно в GeoJSON
electric_f.to_file('electric.geojson', driver='GeoJSON')

In [ ]:
# для позиций, которые есть не во всех случаях
tags = {"landuse": ["construction"]}  # получить стройплощадки
try:
    construction = ox.features.features_from_polygon(boarders.unary_union, tags)  # district.unary_union - возвращает объединённую геометрию
    construction.head()
    construction.reset_index(drop=True, inplace=True) # Сброс индексов в геодатафрейме с удалением старых индексов и сохранением изменений в исходном геодатафрейме
    columns_to_keep_construction = ['landuse', 'name', 'geometry'] # Выбор нужных столбцов (включая geometry)
    construction_f = construction[columns_to_keep_construction] # Создаем новый датафрейм с выбранными столбцами
    construction_f # и выводим его посмотреть
    construction_f.to_file('construction.geojson', driver='GeoJSON') # Сохраняем отфильтрованный датафрейм обратно в GeoJSON

except ox._errors.InsufficientResponseError:
    print("Нет данных по указанному запросу")

In [ ]:
# для позиций, которые есть не во всех случаях
tags = {"railway": ["rail"]}  # получить железные дороги
try:
    railway = ox.features.features_from_polygon(boarders.unary_union, tags)  # district.unary_union - возвращает объединённую геометрию
    railway.head()
    railway.reset_index(drop=True, inplace=True) # Сброс индексов в геодатафрейме с удалением старых индексов и сохранением изменений в исходном геодатафрейме
    columns_to_keep_railway = ['railway', 'name', 'geometry'] # Выбор нужных столбцов (включая geometry)
    railway_f = railway[columns_to_keep_railway] # Создаем новый датафрейм с выбранными столбцами
    railway_f # и выводим его посмотреть
    railway_f.to_file('railway.geojson', driver='GeoJSON') # Сохраняем отфильтрованный датафрейм обратно в GeoJSON

except ox._errors.InsufficientResponseError:
    print("Нет данных по указанному запросу")

In [ ]:
# для позиций, которые есть не во всех случаях
tags = {"place": ["square"]}  # получить площади
try:
    openarea_p = ox.features.features_from_polygon(boarders.unary_union, tags)  # district.unary_union - возвращает объединённую геометрию
    openarea_p.head()
    openarea_p.reset_index(drop=True, inplace=True) # Сброс индексов в геодатафрейме с удалением старых индексов и сохранением изменений в исходном геодатафрейме
    columns_to_keep_openarea_p = ['name', 'geometry'] # Выбор нужных столбцов (включая geometry)
    openarea_p_f = openarea_p[columns_to_keep_openarea_p] # Создаем новый датафрейм с выбранными столбцами
    openarea_p_f # и выводим его посмотреть
    openarea_p_f.to_file('openarea_p.geojson', driver='GeoJSON') # Сохраняем отфильтрованный датафрейм обратно в GeoJSON

except ox._errors.InsufficientResponseError:
    print("Нет данных по указанному запросу")

In [ ]:
# переводим файл с границами в метрическую систему координат для построения буфера
def convert_crs(src_territory, new_crs: int, inplace=False):
    """https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.to_crs.html"""
    return src_territory.to_crs(new_crs, inplace=inplace)

metric_territory = convert_crs(boarders, 32636)
metric_territory.crs

<Projected CRS: EPSG:32636>
Name: WGS 84 / UTM zone 36N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 30°E and 36°E, northern hemisphere between equator and 84°N, onshore and offshore. Belarus. Cyprus. Egypt. Ethiopia. Finland. Israel. Jordan. Kenya. Lebanon. Moldova. Norway. Russian Federation. Saudi Arabia. Sudan. Syria. Türkiye (Turkey). Uganda. Ukraine.
- bounds: (30.0, 0.0, 36.0, 84.0)
Coordinate Operation:
- name: UTM zone 36N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
territory_crs = 32636

metric_boarders = convert_crs(boarders, territory_crs)
metric_boarders.explore(tiles="CartoDB positron")

In [ ]:
# строим буферный полигон - расширяем границы для сбора данных по водным объектам
buffer_distance_100 = 100
buffered_area_100 = metric_boarders.geometry.buffer(buffer_distance_100)
buffered_area_100.explore(tiles="CartoDB positron")

In [ ]:
tags = {"waterway": ["river", "canal"], "natural": ["bay"]}  # получить открытые пространства - реки, каналы, заливы

 # Перевод в метрическую систему -> построение буффера -> обратно в географическую
buffered_area = (
    boarders
    .to_crs(32636)
    .buffer(100)
    .to_crs(4326)
    .union_all()
)

openarea_w = ox.features.features_from_polygon(buffered_area, tags)  # district.unary_union - возвращает объединённую геометрию
openarea_w.head()

geometry boat  \
element id                                                                 
way     78518194  LINESTRING (30.27379 59.96504, 30.27147 59.964...  yes   
        78518195  LINESTRING (30.27166 59.96104, 30.27111 59.961...  yes   
        78518209  LINESTRING (30.28645 59.95354, 30.28533 59.953...  yes   
        78518215  LINESTRING (30.30142 59.97748, 30.29724 59.976...  yes   

                         name       name:de       name:en     name:hr  \
element id                                                              
way     78518194  Малая Невка  Kleine Newka  Little Nevka  Mala Nevka   
        78518195     Ждановка    Schdanowka     Zhdanovka    Ždanovka   
        78518209     Ждановка    Schdanowka     Zhdanovka    Ždanovka   
        78518215  Малая Невка  Kleine Newka  Little Nevka  Mala Nevka   

                      name:ru oneway  \
element id                             
way     78518194  Малая Невка    NaN   
        78518195     Ждановка    yes   
        78518209     Ждановка    yes   
        78518215  Малая Невка    NaN   

                                                      source:oneway waterway  \
element id                                                                     
way     78518194                                                NaN    river   
        78518195  постановление Правительства Санкт-Петербурга о...    river   
        78518209  постановление Правительства Санкт-Петербурга о...    river   
        78518215                                                NaN    river   

                 admin_level        boundary have_riverbank       name:fr  \
element id                                                                  
way     78518194           8  administrative            yes  Petite Nevka   
        78518195         NaN             NaN            NaN           NaN   
        78518209           8  administrative            NaN           NaN   
        78518215         NaN             NaN            yes  Petite Nevka   

                     name:uk  wikidata       wikipedia  
element id                                              
way     78518194  Мала Невка  Q4277131  ru:Малая Невка  
        78518195         NaN       NaN             NaN  
        78518209    Жданівка       NaN             NaN  
        78518215  Мала Невка  Q4277131  ru:Малая Невка

In [ ]:
openarea_w.explore(tiles="CartoDB positron")

In [ ]:
# Сброс индексов в геодатафрейме с удалением старых индексов и сохранением изменений в исходном геодатафрейме
openarea_w.reset_index(drop=True, inplace=True)

# Выбор нужных столбцов (включая geometry)
columns_to_keep_openarea_w = [ 'waterway', 'name', 'geometry']

# Создаем новый датафрейм с выбранными столбцами
openarea_w_f = openarea_w[columns_to_keep_openarea_w]

# и выводим его посмотреть
openarea_w_f

,waterway,name,geometry
0,river,Малая Невка,"LINESTRING (30.27379 59.96504, 30.27147 59.964..."
1,river,Ждановка,"LINESTRING (30.27166 59.96104, 30.27111 59.961..."
2,river,Ждановка,"LINESTRING (30.28645 59.95354, 30.28533 59.953..."
3,river,Малая Невка,"LINESTRING (30.30142 59.97748, 30.29724 59.976..."


In [ ]:
# Сохраняем отфильтрованный датафрейм обратно в GeoJSON
openarea_w_f.to_file('openarea_waterway.geojson', driver='GeoJSON')

После завершения кода все созданные файлы .geojson необходимо скачать, чтобы продолжить работу в QGis.